In [ ]:
%load_ext autoreload
%autoreload 2
# %load_ext autotime|

## This notebook was used to collect the instruct_qa dataset into a dataframe.

In [17]:
model_names = ['alpaca-7b', 'flan-t5-xxl', 'gpt-3.5-turbo', 'llama-2-7b-chat']
datasets = ['hotpot_qa', 'natural_questions', 'topiocqa']
PATH_NAME = ""   # you need to change it to with the path where you keep instruct qa


In [18]:
import pandas as pd 
import json

column_names = ['id', 'dataset', 'model_name', 'question', 'answer', 'ground_truth', 'context', 'human_verdict']
df = pd.DataFrame(columns=column_names)

In [ ]:
import re

data_collector = []


for dataset in datasets:
    for name in model_names: 
        
        
        files = os.listdir(os.path.join(PATH_NAME, 'results', dataset, 'response'))
        
        # print(files)
        
        pattern = f"^{dataset}_validation_c.*{name}_r-gold-passage_prompt.*qa_p-0\.95_t-0\.95_s-0\.jsonl$"
        
        # print(pattern)
        
        selected_file=None
        
        for file in files: 
            if re.match(pattern, file): 
                print(file)
                selected_file = file
                break
     
        # break
        
        path_answers = os.path.join(PATH_NAME, 'results', dataset, 'response',selected_file)
        path_evaluation = os.path.join(PATH_NAME, 'human_eval_annotations', 'correctness', dataset, f"{name}_human_eval_results.json")
        
        print(path_answers) 
        print(path_evaluation)
        print(20*"=")
        
        answers = []

        with open(path_answers, "r") as f:     
            for line in f: 
                answers.append(json.loads(line)) 

        # print(answers)

        with open(path_evaluation, "r") as f: 
            annotations = json.load(f) 


        for answer in answers: 
            # print(answer.keys())
            id_ = answer['id_'] 
            question = answer['question'] 
            llm_answer = answer['response'] 
            ground_truth = '/'.join(a for a in answer['answer']) 
            prompt = answer['prompt'] 
            verdict = None 


            for evaluated_answer in annotations:
                # print(evaluated_answer.keys())
                # break
                if int(id_) == int(evaluated_answer['id_']):
                    
                    if int(id_) == 735: 
                        print(evaluated_answer)
                    
                    verdict=evaluated_answer['annotation'] 
                    

                    data_collector.append({
                        "id" : int(id_), 
                        "model_name" : name, 
                        "dataset": dataset,
                        "question": question, 
                        "answer": llm_answer, 
                        "ground_truth": ground_truth, 
                        "prompt": prompt, 
                        "human_verdict": verdict
                    })

                    # break


In [ ]:
len(data_collector)

In [21]:
df = pd.DataFrame(data_collector)

In [22]:
df.head()

,id,model_name,dataset,question,answer,ground_truth,prompt,human_verdict
0,68,alpaca-7b,hotpot_qa,who is the younger brother of The episode gues...,Brian Doyle-Murray is the younger brother of t...,Bill Murray,Please answer the following question given the...,incorrect
1,263,alpaca-7b,hotpot_qa,"Which character does this protagonist, who sec...",Romeo.,Tybalt,Please answer the following question given the...,incorrect
2,279,alpaca-7b,hotpot_qa,What type of magazine is Eugene Habecker the c...,Christianity Today magazine.,evangelical Christian periodical,Please answer the following question given the...,incorrect
3,317,alpaca-7b,hotpot_qa,"What Kentucky county has a population of 60,31...",Oldham County.,Oldham County,Please answer the following question given the...,correct
4,335,alpaca-7b,hotpot_qa,Mamie Gummer played the role of Nancy Crozier ...,The Good Wife (CBS),CBS,Please answer the following question given the...,correct


In [23]:
df[df['id'] == 735]

,id,model_name,dataset,question,answer,ground_truth,prompt,human_verdict
8,735,alpaca-7b,hotpot_qa,Are both Disney Magazine and The Ring still in...,"Yes, both Disney Magazine and The Ring are sti...",no,Please answer the following question given the...,correct
108,735,flan-t5-xxl,hotpot_qa,Are both Disney Magazine and The Ring still in...,no,no,Please answer the following question given the...,correct
208,735,gpt-3.5-turbo,hotpot_qa,Are both Disney Magazine and The Ring still in...,"No, Disney Magazine ceased publication in Apri...",no,Please answer the following question given the...,correct
308,735,llama-2-7b-chat,hotpot_qa,Are both Disney Magazine and The Ring still in...,"No, both Disney Magazine and The Ring are no l...",no,[INST] <<SYS>>\nPlease answer the following qu...,correct


In [24]:
df.to_csv('instruct_qa_full.csv', index=False)

In [ ]:
df['dataset'].value_counts()

In [ ]:
df[df['dataset'] == 'hotpot_qa']['human_verdict'].value_counts()

In [ ]:
df[df['dataset'] == 'natural_questions']['human_verdict'].value_counts()

In [ ]:
df[df['dataset'] == 'topiocqa']['human_verdict'].value_counts()

In [29]:
df['answer'] = df['answer'].fillna('No answer')

In [30]:
df.to_csv('instruct_qa_full.csv',index=False)